# Wordtracker Data Extraction

### Table Of Contents

1. Import any libraries as required
2. Expand maximum range of rows
3. Understanding general rules about APIs
4. Understanding the Wordtracker API documentation
5. Get Wordtracker overall database stats (US)
6. Get Wordtracker overall database stats (GLOBAL)
7. Define the list of all keywords being called from the Wordtracker API
8. Break down list of keywords into smaller lists, due to Wordtracker API call limitations
9. "For" loop through each engine, and "for" loop through each keyword in each list
10. Create DataFrames of each API result
11. Concat all DataFrames into one
12. Export combined dataframe to work with later on

___

### 1. Import any libraries as required

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import requests
import json
import sklearn
from scipy import stats
from scipy.stats import norm
from sklearn.utils import resample
import pickle

___

### 2. Expand maximum range of rows

May be a good idea to expand the maximum range of rows to view in your Pandas dataframes.<br>
Use <b>pd.set_option()</b>

In [8]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

___

### 3. Understanding general rules about APIs

<b>Request is usually 3 parts:</b><br>
The URL<br>
The header<br>
The parameters

The <b>url</b> is fairly straightforward and is simply the base url as described in the documentation.<br>
The <b>header</b> is a dictionary of key-value pairs.<br>
The <b>parameters</b> are the filters which we wish to pass into the query.

You need to read the <b>documentation</b> from the API to figure out what the rules are to make requests.<br>
Generally, the process is to assign a <b>variable</b> (response) to the <b>requests.get()</b> method.<br>
Then, you use the <b>.json()</b> method to convert the output into a JSON format.<br>

___

### 4. Understanding the Wordtracker API documentation

Here is the documentation for the Wordtracker API: https://wordtracker.github.io/keyword-api/

### Endpoints

<b>/search</b> (raw keyword suggestions for engine/country_code)<br>
<b>/top</b> (keywords for an engine/country_code)<br>
<b>/count</b> (total keywords available for engine/country_code)<br>
<b>/periods</b> (last 12 months as a list)<br>
<b>/stats/keyword</b> (for each engine/country_code)<br>
<b>/stats/link</b> (Majestic link counts)<br>
<b>/fetch</b> (exact search volume for multiple seed terms)<br>

___

### 5. Get Wordtracker overall database stats (US)

#### Total search volume on Google in the US

In [82]:
app_id = 'YOUR_APP_ID' 
app_key = 'YOUR_APP_KEY' 
endpoint = 'stats/keyword'
engine = 'google'
volume = 'total' 
country_code = 'US' 
scale = False 
url = 'https://api.lc.wordtracker.com/v2/' #provided in the documentation

response = requests.get(('{}{}?&engine={}&volume={}&country_code={}&scale={}&&app_id={}&app_key={}').format(url, endpoint, engine, volume, country_code, scale, app_id, app_key))
google_stats = pd.DataFrame(response.json(), index=[0])
google_stats

,max,count,sum,avg,engine,country_code,volume
0,1365152,114231420,594234483,5,google,US,total


#### Total search volume on YouTube in the US

In [83]:
app_id = 'YOUR_APP_ID' 
app_key = 'YOUR_APP_KEY' 
endpoint = 'stats/keyword'
engine = 'youtube'
volume = 'total' 
country_code = 'US' 
scale = False 
url = 'https://api.lc.wordtracker.com/v2/' #provided in the documentation

response = requests.get(('{}{}?&engine={}&volume={}&country_code={}&scale={}&&app_id={}&app_key={}').format(url, endpoint, engine, volume, country_code, scale, app_id, app_key))
youtube_stats = pd.DataFrame(response.json(), index=[0])
youtube_stats

,max,count,sum,avg,engine,country_code,volume
0,66073,9166488,60508218,6,youtube,US,total


#### Total search volume on Amazon in the US

In [84]:
app_id = 'YOUR_APP_ID' 
app_key = 'YOUR_APP_KEY' 
endpoint = 'stats/keyword'
engine = 'amazon'
volume = 'total' 
country_code = 'US' 
scale = False 
url = 'https://api.lc.wordtracker.com/v2/' #provided in the documentation

response = requests.get(('{}{}?&engine={}&volume={}&country_code={}&scale={}&&app_id={}&app_key={}').format(url, endpoint, engine, volume, country_code, scale, app_id, app_key))
amazon_stats = pd.DataFrame(response.json(), index=[0])
amazon_stats

,max,count,sum,avg,engine,country_code,volume
0,15771,1395186,11653715,8,amazon,US,total


#### Total search volume on Ebay in the US (not using this, just out of curiosity)

In [86]:
app_id = 'YOUR_APP_ID' 
app_key = 'YOUR_APP_KEY' 
endpoint = 'stats/keyword'
engine = 'ebay'
volume = 'total' 
country_code = 'US' 
scale = False 
url = 'https://api.lc.wordtracker.com/v2/' #provided in the documentation

response = requests.get(('{}{}?&engine={}&volume={}&country_code={}&scale={}&&app_id={}&app_key={}').format(url, endpoint, engine, volume, country_code, scale, app_id, app_key))
ebay_stats = pd.DataFrame(response.json(), index=[0])
ebay_stats

,max,count,sum,avg,engine,country_code,volume
0,4322,562401,3100869,5,ebay,US,total


#### Export engine stats to CSV

In [87]:
# google_stats.to_csv('google_stats.csv', index=False)
# youtube_stats.to_csv('youtube_stats.csv', index=False)
# amazon_stats.to_csv('amazon_stats.csv', index=False)
# ebay_stats.to_csv('ebay_stats.csv', index=False)

___

### 6. Get Wordtracker overall database stats (GLOBAL)

#### Total search volume on Google in the world

In [4]:
app_id = 'YOUR_APP_ID' 
app_key = 'YOUR_APP_KEY' 
endpoint = 'stats/keyword'
engine = 'google'
volume = 'total' 
country_code = 'GL' 
scale = False 
url = 'https://api.lc.wordtracker.com/v2/' #provided in the documentation

response = requests.get(('{}{}?&engine={}&volume={}&country_code={}&scale={}&&app_id={}&app_key={}').format(url, endpoint, engine, volume, country_code, scale, app_id, app_key))
google_stats_global = pd.DataFrame(response.json(), index=[0])
google_stats_global

,max,count,sum,avg,engine,country_code,volume
0,14317956,158815187,1621049083,10,google,GL,total


#### Total search volume on YouTube in the world

In [5]:
app_id = 'YOUR_APP_ID' 
app_key = 'YOUR_APP_KEY' 
endpoint = 'stats/keyword'
engine = 'youtube'
volume = 'total' 
country_code = 'GL' 
scale = False 
url = 'https://api.lc.wordtracker.com/v2/' #provided in the documentation

response = requests.get(('{}{}?&engine={}&volume={}&country_code={}&scale={}&&app_id={}&app_key={}').format(url, endpoint, engine, volume, country_code, scale, app_id, app_key))
youtube_stats_global = pd.DataFrame(response.json(), index=[0])
youtube_stats_global

,max,count,sum,avg,engine,country_code,volume
0,372540,60572442,731872388,12,youtube,GL,total


#### Total search volume on Amazon in the world

In [6]:
app_id = 'YOUR_APP_ID' 
app_key = 'YOUR_APP_KEY' 
endpoint = 'stats/keyword'
engine = 'amazon'
volume = 'total' 
country_code = 'GL' 
scale = False 
url = 'https://api.lc.wordtracker.com/v2/' #provided in the documentation

response = requests.get(('{}{}?&engine={}&volume={}&country_code={}&scale={}&&app_id={}&app_key={}').format(url, endpoint, engine, volume, country_code, scale, app_id, app_key))
amazon_stats_global = pd.DataFrame(response.json(), index=[0])
amazon_stats_global

,max,count,sum,avg,engine,country_code,volume
0,29962,3650972,27109062,7,amazon,GL,total


#### Total search volume on Ebay in the world (out of curiosity)

In [7]:
app_id = 'YOUR_APP_ID' 
app_key = 'YOUR_APP_KEY' 
endpoint = 'stats/keyword'
engine = 'ebay'
volume = 'total' 
country_code = 'GL' 
scale = False 
url = 'https://api.lc.wordtracker.com/v2/' #provided in the documentation

response = requests.get(('{}{}?&engine={}&volume={}&country_code={}&scale={}&&app_id={}&app_key={}').format(url, endpoint, engine, volume, country_code, scale, app_id, app_key))
ebay_stats_global = pd.DataFrame(response.json(), index=[0])
ebay_stats_global

,max,count,sum,avg,engine,country_code,volume
0,10094,1630816,10223581,6,ebay,GL,total


#### Export engine stats to CSV

In [8]:
# google_stats_global.to_csv('google_stats_global.csv', index=False)
# youtube_stats_global.to_csv('youtube_stats_global.csv', index=False)
# amazon_stats_global.to_csv('amazon_stats_global.csv', index=False)
# ebay_stats_global.to_csv('ebay_stats_global.csv', index=False)

___

### 7. Define the list of all keywords being called from the Wordtracker API

In [2]:
# We will pull volume data from Wordtracker using this list of keywords.
# We generated this list by searching “athleisure” on Amazon and used common keywords found in "featured" results.

word_list = [
"yoga pants"
,"sweatpants"
,"sweatshirt"
,"sweatband"
,"windbreaker"
,"leggings"
,"tank top"
,"muscle tank"
,"long sleeve"
,"short sleeve"
,"crew neck"
,"thumb hole"
,"pullover"
,"fleece"
,"joggers"
,"hoodie"
,"hooded"
,"capri"
,"muscle tee"
,"basic tee"
,"mesh"
,"striped"
,"stripes"
,"3 stripes"
,"stripe"
,"stretch"
,"stretchy"
,"stretchable"
,"flex"
,"flexing"
,"lightweight"
,"spandex"
,"breathable"
,"loose"
,"loose fit"
,"fitted"
,"core"
,"blend"
,"cotton"
,"high waist"
,"tights"
,"baggy"
,"slim"
,"activewear"
,"sleeveless"
,"active"
,"athletic"
,"relaxed"
,"performance"
,"pockets"
,"drawstring"
,"squat"
,"tummy"
,"movement"
,"skinny"
,"workout"
,"racerback"
,"scoop neck"
,"v neck"
,"raglan"
,"tapered"
,"lined"
,"quick dry"
,"padded"
,"running"
,"ventilated"
,"warm up"
,"crop top"
,"crop hoodie"
,"elastic"
,"dri fit"
,"cropped"
,"wicking"
,"mid rise"
,"active gear"
,"running gear"
,"split back"]

In [3]:
#how long is our list of keywords?
len(word_list)

77

___

### 8. Break down list of keywords into smaller lists, due to Wordtracker API call limitations

In [58]:
# The Wordtracker API says we can make up to 200 calls during the "free trial" period.
# However in practce, we were denied access to the API after approximately 100 calls.
# We have to call 77 keywords x 3 engines = 231 calls.
# When we "for" loop through all 231 calls, the calls will stop partway through. 
# So, to get around this limitation imposed by the Wordtracker API, we will break our 77 keyword list down.
# To be careful, we will break the list down into 24 smaller lists of 3.
# NOTE - we also tested a larger list size for "search_terms_2" (10 keywords) just out of curiosity.

search_terms_1 = [
"yoga pants"
,"sweatpants"
,"sweatshirt"
]

search_terms_2 = [
"crew neck"
,"thumb hole"
,"pullover"
,"fleece"
,"joggers"
,"hoodie"
,"hooded"
,"capri"
,"muscle tee"
,"basic tee"
]

search_terms_3 = [
"sweatband"
,"windbreaker"
,"leggings"
]
search_terms_4 = [
"tank top"
,"muscle tank"
,"long sleeve"
]

search_terms_5 = [
"short sleeve"
,"mesh"
,"striped"
]
search_terms_6 = [
"striped"
,"stripes"
,"3 stripes"
]
search_terms_7 = [
"stripe"
,"stretch"
,"stretchy"
]
search_terms_8 = [
"stretchable"
,"flex"
,"flexing"
]
search_terms_9 = [
"lightweight"
,"spandex"
,"breathable"
]
search_terms_10 = [
"loose"
,"loose fit"
,"fitted"
]
search_terms_11 = [
"core"
,"blend"
,"cotton"
]
search_terms_12 = [
"high waist"
,"tights"
,"baggy"
]
search_terms_13 = [
"slim"
,"activewear"
,"sleeveless"
]
search_terms_14 = [
"active"
,"athletic"
,"relaxed"
]
search_terms_15 = [
"performance"
,"pockets"
,"drawstring"
]
search_terms_16 = [
"squat"
,"tummy"
,"movement"
]
search_terms_17 = [
"skinny"
,"workout"
,"racerback"
]
search_terms_18 = [
"scoop neck"
,"v neck"
,"raglan"
]
search_terms_19 = [
"tapered"
,"lined"
,"quick dry"
]
search_terms_20 = [
"padded"
,"running"
,"ventilated"
]
search_terms_21 = [
"warm up"
,"crop top"
,"crop hoodie"
]
search_terms_22 = [
"elastic"
,"dri fit"
,"cropped"
]
search_terms_23 = [
"wicking"
,"mid rise"
,"active gear"
]
search_terms_24 = [
"running gear"
,"split back"
]

___

### 9. "For" loop through each engine, and "for" loop through each keyword in each list

In [97]:
#NOTE - this code was run 24 times, 1 time for each "search_terms_#" list defined above.

# Initialize empty lists of column values
keyword = []
volume = []
year = []
months = []
search_engine = []
total_volume_for_last_12_months = []
avg_volume_for_last_12_months = []
total_months_seen = []

# set the list of engines that you want to call keywords from
engine_list = ["google", "youtube", "amazon"]

# set the list of terms that you want to call from the API using search_terms_1 through search_terms_24
word_list = search_terms_24

#for loop to go through each engine
for engine in engine_list:
    
    # for loop to go through each keyword in each list
    for word in word_list:

        # Set up the parameters we want to pass to the API.
        parameters = {"seeds": word, 
                      "engine": engine, 
                      "country_code": "US", 
                      "type": "phrase",
                      "sort": "total",
                      "start":1, 
                      "limit": 1, 
                      "app_id": "YOUR APP_ID", 
                      "app_key": "YOUR APP_KEY" }
        
        # Make a get request with the parameters.
        response = requests.get("https://api.lc.wordtracker.com/v2/search?", params=parameters)
        
        # Store constant volume values found in outer layer of dictionary
        if len(response.json()['results']) != 0:
            
            print(response.json()['results'][0])
            total_volume = response.json()['results'][0]['total_volume_for_last_12_months']
            avg_volume = response.json()['results'][0]['avg_volume_for_last_12_months']
            total_months = response.json()['results'][0]['total_months_seen']
            
            # Append values to pre-defined column lists
            for month in response.json()['results'][0]['volume_by_month']:
                keyword.append(word)
                volume.append(month['volume'])
                year.append(month['year'])
                months.append(month['month'])
                search_engine.append(engine)
                total_volume_for_last_12_months.append(total_volume)
                avg_volume_for_last_12_months.append(avg_volume)
                total_months_seen.append(total_months)
                
        #else: continue statement in case a response does not have content, we want the "for" loop to continue.
        else:
            continue

{'keyword': 'running gear', 'volume_by_month': [{'volume': 0, 'year': '2018', 'month': '06'}, {'volume': 6, 'year': '2018', 'month': '07'}, {'volume': 2, 'year': '2018', 'month': '08'}, {'volume': 2, 'year': '2018', 'month': '09'}, {'volume': 9, 'year': '2018', 'month': '10'}, {'volume': 7, 'year': '2018', 'month': '11'}, {'volume': 7, 'year': '2018', 'month': '12'}, {'volume': 4, 'year': '2019', 'month': '01'}, {'volume': 8, 'year': '2019', 'month': '02'}, {'volume': 1, 'year': '2019', 'month': '03'}, {'volume': 9, 'year': '2019', 'month': '04'}, {'volume': 7, 'year': '2019', 'month': '05'}], 'panelist_ids_by_month': [{'ids': 0, 'year': '2018', 'month': '06'}, {'ids': 3, 'year': '2018', 'month': '07'}, {'ids': 1, 'year': '2018', 'month': '08'}, {'ids': 1, 'year': '2018', 'month': '09'}, {'ids': 4, 'year': '2018', 'month': '10'}, {'ids': 4, 'year': '2018', 'month': '11'}, {'ids': 7, 'year': '2018', 'month': '12'}, {'ids': 3, 'year': '2019', 'month': '01'}, {'ids': 6, 'year': '2019', 'm

___

### 10. Create DataFrames of each API result

#### DataFrame 1

In [42]:
# DataFrame creation using lists of each column
data_1 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_1 = pd.DataFrame.from_dict(data_1)
# athleisure_df_1.to_csv('athleisure_df_1.csv')
athleisure_df_1

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,yoga pants,0,2018,06,google,779,65,12
1,yoga pants,68,2018,07,google,779,65,12
2,yoga pants,107,2018,08,google,779,65,12
3,yoga pants,63,2018,09,google,779,65,12
4,yoga pants,72,2018,10,google,779,65,12
5,yoga pants,77,2018,11,google,779,65,12
6,yoga pants,56,2018,12,google,779,65,12
7,yoga pants,50,2019,01,google,779,65,12
8,yoga pants,42,2019,02,google,779,65,12
9,yoga pants,64,2019,03,google,779,65,12


#### DataFrame 2

In [44]:
# DataFrame creation using lists of each column
data_2 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_2 = pd.DataFrame.from_dict(data_2)
# athleisure_df_2.to_csv('athleisure_df_2.csv')
athleisure_df_2

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,crew neck,0,2018,06,google,353,29,12
1,crew neck,41,2018,07,google,353,29,12
2,crew neck,41,2018,08,google,353,29,12
3,crew neck,54,2018,09,google,353,29,12
4,crew neck,36,2018,10,google,353,29,12
5,crew neck,31,2018,11,google,353,29,12
6,crew neck,20,2018,12,google,353,29,12
7,crew neck,36,2019,01,google,353,29,12
8,crew neck,22,2019,02,google,353,29,12
9,crew neck,20,2019,03,google,353,29,12


#### DataFrame 3

In [48]:
# DataFrame creation using lists of each column
data_3 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_3 = pd.DataFrame.from_dict(data_3)
# athleisure_df_3.to_csv('athleisure_df_3.csv')
athleisure_df_3

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,sweatband,0,2018,06,google,92,8,12
1,sweatband,15,2018,07,google,92,8,12
2,sweatband,12,2018,08,google,92,8,12
3,sweatband,9,2018,09,google,92,8,12
4,sweatband,5,2018,10,google,92,8,12
5,sweatband,7,2018,11,google,92,8,12
6,sweatband,4,2018,12,google,92,8,12
7,sweatband,7,2019,01,google,92,8,12
8,sweatband,7,2019,02,google,92,8,12
9,sweatband,7,2019,03,google,92,8,12


#### DataFrame 4

In [50]:
# DataFrame creation using lists of each column
data_4 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_4 = pd.DataFrame.from_dict(data_4)
# athleisure_df_4.to_csv('athleisure_df_4.csv')
athleisure_df_4

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,tank top,0,2018,06,google,654,55,12
1,tank top,89,2018,07,google,654,55,12
2,tank top,58,2018,08,google,654,55,12
3,tank top,49,2018,09,google,654,55,12
4,tank top,56,2018,10,google,654,55,12
5,tank top,39,2018,11,google,654,55,12
6,tank top,37,2018,12,google,654,55,12
7,tank top,61,2019,01,google,654,55,12
8,tank top,55,2019,02,google,654,55,12
9,tank top,55,2019,03,google,654,55,12


#### DataFrame 5

In [54]:
# DataFrame creation using lists of each column
data_5 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_5 = pd.DataFrame.from_dict(data_5)
# athleisure_df_5.to_csv('athleisure_df_5.csv')
athleisure_df_5

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,short sleeve,0,2018,06,google,124,10,12
1,short sleeve,19,2018,07,google,124,10,12
2,short sleeve,14,2018,08,google,124,10,12
3,short sleeve,7,2018,09,google,124,10,12
4,short sleeve,9,2018,10,google,124,10,12
5,short sleeve,5,2018,11,google,124,10,12
6,short sleeve,3,2018,12,google,124,10,12
7,short sleeve,7,2019,01,google,124,10,12
8,short sleeve,12,2019,02,google,124,10,12
9,short sleeve,9,2019,03,google,124,10,12


#### DataFrame 6

In [57]:
# DataFrame creation using lists of each column
data_6 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_6 = pd.DataFrame.from_dict(data_6)
# athleisure_df_6.to_csv('athleisure_df_6.csv')
athleisure_df_6

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,striped,0,2018,06,google,2260,188,12
1,striped,25,2018,07,google,2260,188,12
2,striped,43,2018,08,google,2260,188,12
3,striped,76,2018,09,google,2260,188,12
4,striped,118,2018,10,google,2260,188,12
5,striped,55,2018,11,google,2260,188,12
6,striped,151,2018,12,google,2260,188,12
7,striped,255,2019,01,google,2260,188,12
8,striped,332,2019,02,google,2260,188,12
9,striped,404,2019,03,google,2260,188,12


#### DataFrame 7

In [60]:
# DataFrame creation using lists of each column
data_7 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_7 = pd.DataFrame.from_dict(data_7)
# athleisure_df_7.to_csv('athleisure_df_7.csv')
athleisure_df_7

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,stripe,0,2018,06,google,6365,530,12
1,stripe,636,2018,07,google,6365,530,12
2,stripe,738,2018,08,google,6365,530,12
3,stripe,772,2018,09,google,6365,530,12
4,stripe,757,2018,10,google,6365,530,12
5,stripe,479,2018,11,google,6365,530,12
6,stripe,286,2018,12,google,6365,530,12
7,stripe,474,2019,01,google,6365,530,12
8,stripe,425,2019,02,google,6365,530,12
9,stripe,429,2019,03,google,6365,530,12


#### DataFrame 8

In [62]:
# DataFrame creation using lists of each column
data_8 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_8 = pd.DataFrame.from_dict(data_8)
# athleisure_df_8.to_csv('athleisure_df_8.csv')
athleisure_df_8

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,stretchable,0,2018,06,google,7,1,3
1,stretchable,0,2018,07,google,7,1,3
2,stretchable,0,2018,08,google,7,1,3
3,stretchable,0,2018,09,google,7,1,3
4,stretchable,0,2018,10,google,7,1,3
5,stretchable,3,2018,11,google,7,1,3
6,stretchable,0,2018,12,google,7,1,3
7,stretchable,0,2019,01,google,7,1,3
8,stretchable,0,2019,02,google,7,1,3
9,stretchable,0,2019,03,google,7,1,3


#### DataFrame 9

In [64]:
# DataFrame creation using lists of each column
data_9 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_9 = pd.DataFrame.from_dict(data_9)
# athleisure_df_9.to_csv('athleisure_df_9.csv')
athleisure_df_9

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,lightweight,0,2018,06,google,363,30,12
1,lightweight,29,2018,07,google,363,30,12
2,lightweight,39,2018,08,google,363,30,12
3,lightweight,34,2018,09,google,363,30,12
4,lightweight,59,2018,10,google,363,30,12
5,lightweight,31,2018,11,google,363,30,12
6,lightweight,17,2018,12,google,363,30,12
7,lightweight,24,2019,01,google,363,30,12
8,lightweight,26,2019,02,google,363,30,12
9,lightweight,25,2019,03,google,363,30,12


#### DataFrame 10

In [66]:
# DataFrame creation using lists of each column
data_10 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_10 = pd.DataFrame.from_dict(data_10)
# athleisure_df_10.to_csv('athleisure_df_10.csv')
athleisure_df_10

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,loose,0,2018,06,google,1458,121,12
1,loose,143,2018,07,google,1458,121,12
2,loose,95,2018,08,google,1458,121,12
3,loose,108,2018,09,google,1458,121,12
4,loose,118,2018,10,google,1458,121,12
5,loose,118,2018,11,google,1458,121,12
6,loose,94,2018,12,google,1458,121,12
7,loose,137,2019,01,google,1458,121,12
8,loose,132,2019,02,google,1458,121,12
9,loose,144,2019,03,google,1458,121,12


#### DataFrame 11

In [68]:
# DataFrame creation using lists of each column
data_11 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_11 = pd.DataFrame.from_dict(data_11)
# athleisure_df_11.to_csv('athleisure_df_11.csv')
athleisure_df_11

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,core,0,2018,06,google,6899,575,12
1,core,31,2018,07,google,6899,575,12
2,core,64,2018,08,google,6899,575,12
3,core,115,2018,09,google,6899,575,12
4,core,136,2018,10,google,6899,575,12
5,core,181,2018,11,google,6899,575,12
6,core,952,2018,12,google,6899,575,12
7,core,1123,2019,01,google,6899,575,12
8,core,1206,2019,02,google,6899,575,12
9,core,1179,2019,03,google,6899,575,12


#### DataFrame 12

In [73]:
# DataFrame creation using lists of each column
data_12 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_12 = pd.DataFrame.from_dict(data_12)
# athleisure_df_12.to_csv('athleisure_df_12.csv')
athleisure_df_12

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,high waist,0,2018,06,google,29,2,9
1,high waist,4,2018,07,google,29,2,9
2,high waist,4,2018,08,google,29,2,9
3,high waist,4,2018,09,google,29,2,9
4,high waist,2,2018,10,google,29,2,9
5,high waist,5,2018,11,google,29,2,9
6,high waist,0,2018,12,google,29,2,9
7,high waist,0,2019,01,google,29,2,9
8,high waist,3,2019,02,google,29,2,9
9,high waist,4,2019,03,google,29,2,9


#### DataFrame 13

In [75]:
# DataFrame creation using lists of each column
data_13 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_13 = pd.DataFrame.from_dict(data_13)
# athleisure_df_13.to_csv('athleisure_df_13.csv')
athleisure_df_13

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,slim,0,2018,06,google,1215,101,12
1,slim,106,2018,07,google,1215,101,12
2,slim,101,2018,08,google,1215,101,12
3,slim,126,2018,09,google,1215,101,12
4,slim,138,2018,10,google,1215,101,12
5,slim,170,2018,11,google,1215,101,12
6,slim,101,2018,12,google,1215,101,12
7,slim,124,2019,01,google,1215,101,12
8,slim,94,2019,02,google,1215,101,12
9,slim,58,2019,03,google,1215,101,12


#### DataFrame 14

In [77]:
# DataFrame creation using lists of each column
data_14 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_14 = pd.DataFrame.from_dict(data_14)
# athleisure_df_14.to_csv('athleisure_df_14.csv')
athleisure_df_14

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,active,0,2018,06,google,1544,129,12
1,active,19,2018,07,google,1544,129,12
2,active,35,2018,08,google,1544,129,12
3,active,103,2018,09,google,1544,129,12
4,active,149,2018,10,google,1544,129,12
5,active,75,2018,11,google,1544,129,12
6,active,338,2018,12,google,1544,129,12
7,active,213,2019,01,google,1544,129,12
8,active,170,2019,02,google,1544,129,12
9,active,183,2019,03,google,1544,129,12


#### DataFrame 15

In [79]:
# DataFrame creation using lists of each column
data_15 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_15 = pd.DataFrame.from_dict(data_15)
# athleisure_df_15.to_csv('athleisure_df_15.csv')
athleisure_df_15

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,performance,0,2018,06,google,1656,138,12
1,performance,139,2018,07,google,1656,138,12
2,performance,124,2018,08,google,1656,138,12
3,performance,126,2018,09,google,1656,138,12
4,performance,147,2018,10,google,1656,138,12
5,performance,99,2018,11,google,1656,138,12
6,performance,100,2018,12,google,1656,138,12
7,performance,185,2019,01,google,1656,138,12
8,performance,124,2019,02,google,1656,138,12
9,performance,160,2019,03,google,1656,138,12


#### DataFrame 16

In [81]:
# DataFrame creation using lists of each column
data_16 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_16 = pd.DataFrame.from_dict(data_16)
# athleisure_df_16.to_csv('athleisure_df_16.csv')
athleisure_df_16

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,squat,0,2018,06,google,606,50,12
1,squat,62,2018,07,google,606,50,12
2,squat,68,2018,08,google,606,50,12
3,squat,54,2018,09,google,606,50,12
4,squat,79,2018,10,google,606,50,12
5,squat,48,2018,11,google,606,50,12
6,squat,43,2018,12,google,606,50,12
7,squat,54,2019,01,google,606,50,12
8,squat,50,2019,02,google,606,50,12
9,squat,47,2019,03,google,606,50,12


#### DataFrame 17

In [83]:
# DataFrame creation using lists of each column
data_17 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_17 = pd.DataFrame.from_dict(data_17)
# athleisure_df_17.to_csv('athleisure_df_17.csv')
athleisure_df_17

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,skinny,0,2018,06,google,683,57,12
1,skinny,135,2018,07,google,683,57,12
2,skinny,72,2018,08,google,683,57,12
3,skinny,67,2018,09,google,683,57,12
4,skinny,63,2018,10,google,683,57,12
5,skinny,46,2018,11,google,683,57,12
6,skinny,23,2018,12,google,683,57,12
7,skinny,54,2019,01,google,683,57,12
8,skinny,43,2019,02,google,683,57,12
9,skinny,31,2019,03,google,683,57,12


#### DataFrame 18

In [86]:
# DataFrame creation using lists of each column
data_18 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_18 = pd.DataFrame.from_dict(data_18)
# athleisure_df_18.to_csv('athleisure_df_18.csv')
athleisure_df_18

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,scoop neck,0,2018,06,google,95,8,12
1,scoop neck,12,2018,07,google,95,8,12
2,scoop neck,19,2018,08,google,95,8,12
3,scoop neck,16,2018,09,google,95,8,12
4,scoop neck,7,2018,10,google,95,8,12
5,scoop neck,5,2018,11,google,95,8,12
6,scoop neck,5,2018,12,google,95,8,12
7,scoop neck,4,2019,01,google,95,8,12
8,scoop neck,5,2019,02,google,95,8,12
9,scoop neck,2,2019,03,google,95,8,12


#### DataFrame 19

In [88]:
# DataFrame creation using lists of each column
data_19 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_19 = pd.DataFrame.from_dict(data_19)
# athleisure_df_19.to_csv('athleisure_df_19.csv')
athleisure_df_19

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,tapered,0,2018,06,google,238,20,12
1,tapered,27,2018,07,google,238,20,12
2,tapered,25,2018,08,google,238,20,12
3,tapered,16,2018,09,google,238,20,12
4,tapered,32,2018,10,google,238,20,12
5,tapered,24,2018,11,google,238,20,12
6,tapered,7,2018,12,google,238,20,12
7,tapered,26,2019,01,google,238,20,12
8,tapered,16,2019,02,google,238,20,12
9,tapered,14,2019,03,google,238,20,12


#### DataFrame 20

In [90]:
# DataFrame creation using lists of each column
data_20 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_20 = pd.DataFrame.from_dict(data_20)
# athleisure_df_20.to_csv('athleisure_df_20.csv')
athleisure_df_20

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,padded,0,2018,06,google,83,7,12
1,padded,8,2018,07,google,83,7,12
2,padded,6,2018,08,google,83,7,12
3,padded,9,2018,09,google,83,7,12
4,padded,16,2018,10,google,83,7,12
5,padded,9,2018,11,google,83,7,12
6,padded,7,2018,12,google,83,7,12
7,padded,7,2019,01,google,83,7,12
8,padded,3,2019,02,google,83,7,12
9,padded,6,2019,03,google,83,7,12


#### DataFrame 21

In [92]:
# DataFrame creation using lists of each column
data_21 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_21 = pd.DataFrame.from_dict(data_21)
# athleisure_df_21.to_csv('athleisure_df_21.csv')
athleisure_df_21

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,warm up,0,2018,06,google,349,29,12
1,warm up,12,2018,07,google,349,29,12
2,warm up,12,2018,08,google,349,29,12
3,warm up,22,2018,09,google,349,29,12
4,warm up,18,2018,10,google,349,29,12
5,warm up,9,2018,11,google,349,29,12
6,warm up,30,2018,12,google,349,29,12
7,warm up,60,2019,01,google,349,29,12
8,warm up,48,2019,02,google,349,29,12
9,warm up,45,2019,03,google,349,29,12


#### DataFrame 22

In [94]:
# DataFrame creation using lists of each column
data_22 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_22 = pd.DataFrame.from_dict(data_22)
# athleisure_df_22.to_csv('athleisure_df_22.csv')
athleisure_df_22

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,elastic,0,2018,06,google,1291,108,12
1,elastic,35,2018,07,google,1291,108,12
2,elastic,74,2018,08,google,1291,108,12
3,elastic,128,2018,09,google,1291,108,12
4,elastic,147,2018,10,google,1291,108,12
5,elastic,87,2018,11,google,1291,108,12
6,elastic,99,2018,12,google,1291,108,12
7,elastic,195,2019,01,google,1291,108,12
8,elastic,120,2019,02,google,1291,108,12
9,elastic,114,2019,03,google,1291,108,12


#### DataFrame 23

In [96]:
# DataFrame creation using lists of each column
data_23 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_23 = pd.DataFrame.from_dict(data_23)
# athleisure_df_23.to_csv('athleisure_df_23.csv')
athleisure_df_23

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,wicking,0,2018,06,google,100,8,11
1,wicking,12,2018,07,google,100,8,11
2,wicking,12,2018,08,google,100,8,11
3,wicking,0,2018,09,google,100,8,11
4,wicking,7,2018,10,google,100,8,11
5,wicking,3,2018,11,google,100,8,11
6,wicking,3,2018,12,google,100,8,11
7,wicking,26,2019,01,google,100,8,11
8,wicking,13,2019,02,google,100,8,11
9,wicking,7,2019,03,google,100,8,11


#### DataFrame 24

In [98]:
# DataFrame creation using lists of each column
data_24 = {'keyword': keyword, 
        'volume': volume, 
        'year': year, 
        'month': months, 
        'engine': search_engine,
        'total_volume_for_last_12_months': total_volume_for_last_12_months,
        'avg_volume_for_last_12_months': avg_volume_for_last_12_months,
        'total_months_seen': total_months_seen}

athleisure_df_24 = pd.DataFrame.from_dict(data_24)
# athleisure_df_24.to_csv('athleisure_df_24.csv')
athleisure_df_24

,keyword,volume,year,month,engine,total_volume_for_last_12_months,avg_volume_for_last_12_months,total_months_seen
0,running gear,0,2018,06,google,65,5,12
1,running gear,6,2018,07,google,65,5,12
2,running gear,2,2018,08,google,65,5,12
3,running gear,2,2018,09,google,65,5,12
4,running gear,9,2018,10,google,65,5,12
5,running gear,7,2018,11,google,65,5,12
6,running gear,7,2018,12,google,65,5,12
7,running gear,4,2019,01,google,65,5,12
8,running gear,8,2019,02,google,65,5,12
9,running gear,1,2019,03,google,65,5,12


___

### 11. Concat all DataFrames into one

In [ ]:
athleisure_df = pd.concat([
athleisure_df_1
,athleisure_df_2
,athleisure_df_3
,athleisure_df_4
,athleisure_df_5
,athleisure_df_6
,athleisure_df_7
,athleisure_df_8
,athleisure_df_9
,athleisure_df_10
,athleisure_df_11
,athleisure_df_12
,athleisure_df_13
,athleisure_df_14
,athleisure_df_15
,athleisure_df_16
,athleisure_df_17
,athleisure_df_18
,athleisure_df_19
,athleisure_df_20
,athleisure_df_21
,athleisure_df_22
,athleisure_df_23
,athleisure_df_24
])

___

### 12. Export combined dataframe to work with later on

In [104]:
# export the combined dataframe
# athleisure_df.to_csv('athleisure.csv')